$NicheVI$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

import anndata as ad
import numpy as np
import pandas as pd

from rich import print


from sklearn.neighbors import NearestNeighbors


import scvi

In [3]:
!ls /home/nathanlevy/Data/

adata_MERFISH_24w.h5ad	adata_scvi_merfish4w.h5ad


In [4]:
data_dir = "/home/nathanlevy/Data/"
file = "adata_scvi_merfish4w.h5ad"

In [5]:
adata = ad.read_h5ad(data_dir + file)
print(adata)

AnnData object with n_obs × n_vars = 123632 × 374
    obs: 'fov', 'center_x', 'center_y', 'min_x', 'max_x', 'min_y', 'max_y', 'age', 'clust_annot', 'slice', 
'organism_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'cell_type_ontology_term_id', 
'assay_ontology_term_id', 'tissue_ontology_term_id', 'disease_ontology_term_id', 
'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'is_primary_data', 
'cell_type_annot', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 
'development_stage', 'donor_slice', 'n_counts', 'cell_index', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'cell_type_colors', 'donor_id_colors', 'log1p', 'neighbors', 'pca', 
'umap'
    obsm: 'X_pca', 'X_scVI', 'X_spVI', 'X_spatial', 'X_umap', 'centroids', 'neighborhood_composition', 'spatial'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

## First train (linear)scVI

In [6]:
scvi.model.LinearSCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor_id",
)


vae = scvi.model.LinearSCVI(
    adata,
    gene_likelihood="poisson",
    n_layers=1,
    n_latent=10,
)

vae.train(
    max_epochs=2,
    train_size=0.8,
    validation_size=0.2,
    plan_kwargs=dict(n_epochs_kl_warmup=80),
    early_stopping=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 2/2: 100%|██████████| 2/2 [00:08<00:00,  3.99s/it, v_num=1, train_loss_step=201, train_loss_epoch=205]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 2/2: 100%|██████████| 2/2 [00:08<00:00,  4.02s/it, v_num=1, train_loss_step=201, train_loss_epoch=205]


In [7]:
adata.obsm["qz1_m"], adata.obsm["qz1_var"] = vae.get_latent_representation(
    return_dist=True
)

## Then train $nicheVI$

In [8]:
k_nn = 10
n_latent_z1 = 10

In [40]:
scvi.model.nicheSCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor_id",
    labels_key="cell_type",
    niche_composition_key="neighborhood_composition",
    niche_indexes_key="niche_indexes",
    sample_key="donor_slice",
    cell_coordinates_key="centroids",
    k_nn=10,
    latent_mean_key="qz1_m",
    latent_var_key="qz1_var",
)

nichevae = scvi.model.nicheSCVI(
    adata,
    # k_nn=k_nn,
    # n_latent_z1=n_latent_z1,
    niche_kl_weight=1,
    niche_components='cell_type',
    gene_likelihood="poisson",
    n_layers=1,
    n_latent=10,
)

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [32]:
adata.obs.reset_index()
adata.obs.cell_type[adata.obsm["niche_indexes"][-1]].value_counts()/10

cell_type
neuron                            0.5
astrocyte                         0.2
inhibitory interneuron            0.2
endothelial cell                  0.1
cell                              0.0
T cell                            0.0
oligodendrocyte                   0.0
microglial cell                   0.0
macrophage                        0.0
pericyte                          0.0
oligodendrocyte precursor cell    0.0
medium spiny neuron               0.0
vascular leptomeningeal cell      0.0
Name: count, dtype: float64

In [11]:
adata.obsm["niche_indexes"]

array([[    25,     17,     30, ...,     39,     42,     19],
       [    34,     38,     37, ...,     40,     30,      2],
       [    40,     19,     35, ...,      4,     18,     23],
       ...,
       [123389, 123409, 123627, ..., 123397, 123628, 123414],
       [123624, 123631, 123598, ..., 123599, 123628, 123582],
       [123598, 123594, 123577, ..., 123619, 123584, 123602]])

In [12]:
adata.obsm["neighborhood_composition"]

,endothelial cell,oligodendrocyte precursor cell,neuron,astrocyte,inhibitory interneuron,pericyte,oligodendrocyte,microglial cell,vascular leptomeningeal cell,medium spiny neuron,macrophage,T cell,cell
275820748261104984743470052142490986087,0.2,0.2,0.3,0.0,0.2,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
302375575077769376592971920558356036886,0.1,0.1,0.7,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
267807829680073723631714599566507550894,0.1,0.2,0.5,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
51672646721370271404472036583758197146,0.1,0.0,0.5,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0
124445072923359013299932652711178203190,0.1,0.0,0.6,0.2,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201935776857989535834640798360655008792,0.1,0.0,0.1,0.7,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
96774665042238935375441098896984824742,0.2,0.0,0.1,0.4,0.1,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0
63916806179766392947488205259727089290,0.2,0.0,0.0,0.7,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0
282657524773287188112567162213233721197,0.0,0.0,0.3,0.3,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0


Experiments with **Niche decoder**

In [13]:
nichevae.summary_stats

n_batch: 4
n_cells: 123632
n_extra_categorical_covs: 0
n_extra_continuous_covs: 0
n_labels: 13
n_latent_mean: 10
n_latent_var: 10
n_niche_composition: 13
n_niche_indexes: 10
n_vars: 374

In [14]:
print(nichevae.module.niche_decoder)

NicheDecoder(
  (decoder): FCLayers(
    (fc_layers): Sequential(
      (Layer 0): Sequential(
        (0): Linear(in_features=10, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): None
        (3): ReLU()
        (4): None
      )
    )
  )
  (mean_decoder): Linear(in_features=128, out_features=100, bias=True)
  (var_decoder): Linear(in_features=128, out_features=100, bias=True)
)

In [15]:
adata.obsm["niche_indexes"][0]

array([25, 17, 30, 43, 10, 36, 35, 39, 42, 19])

In [16]:
nichevae.device

device(type='cpu')

In [41]:
latent_cell = torch.ones((2, 10), device=nichevae.device)  # (batch_size*latent_size)

niche_mean, niche_var = nichevae.module.niche_decoder(
    latent_cell
)  # (batch_size*latent_size*k_nn)

In [37]:
batch_size = 2
latent_size = 10
k_nn = 10

a=k_nn
b=latent_size

# Assuming 'tensor' is your tensor of shape (batch_size, a, b)
# Assuming 'weights' is your tensor of shape (batch_size, a)
tensor = torch.randn(batch_size, a, b)
weights = torch.randn(batch_size, a)

# Multiply the tensor by the weights element-wise
weighted_tensor = tensor * weights.unsqueeze(-1)

# Sum along the 'a' dimension
sum_a = weighted_tensor.sum(dim=1)


In [39]:
niche_mean_mat.shape

torch.Size([2, 10, 10])

In [48]:
# Multiply the tensor by the weights element-wise
weighted_tensor = niche_mean_mat * torch.tensor(adata.obsm["neighborhood_composition"][:2].values).unsqueeze(-1)

# Sum along the 'a' dimension
sum_a = weighted_tensor.sum(dim=1)

In [45]:
nichevae.module.n_niche_components

13

In [46]:
nichevae.module.n_niche_components

niche_mean_mat = niche_mean.reshape(niche_mean.shape[0], nichevae.module.n_niche_components, n_latent_z1)
niche_var_mat = niche_var.reshape(niche_var.shape[0], nichevae.module.n_niche_components, n_latent_z1)

niche_mean_agg = torch.sum(niche_mean_mat, dim=1)
niche_var_agg = torch.sum(niche_var_mat, dim=1)

In [20]:
niche_mean_agg.shape

torch.Size([2, 10])

In [21]:
torch.distributions.normal.Normal(niche_mean_mat, niche_var_mat)

Normal(loc: torch.Size([2, 10, 10]), scale: torch.Size([2, 10, 10]))

In [22]:
nichevae.train(
    max_epochs=1,
    train_size=0.8,
    validation_size=0.2,
    plan_kwargs=dict(n_epochs_kl_warmup=80),
    early_stopping=False,
    batch_size=1000,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1:   0%|          | 0/1 [00:00<?, ?it/s]

/home/nathanlevy/mambaforge/envs/phenospace_23/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('niche_kl_train', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Epoch 1/1: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s, v_num=1, train_loss_step=199, train_loss_epoch=240]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s, v_num=1, train_loss_step=199, train_loss_epoch=240]


## Code to train compo-scVI

In [23]:
scvi.model.compoSCVI.setup_anndata(
    adata,
    layer="counts",
    batch_key="donor_id",
    niche_composition_key="neighborhood_composition",
)

n_cell_types = adata.obsm["neighborhood_composition"].shape[1]

compovae = scvi.model.compoSCVI(
    adata,
    gene_likelihood="poisson",
    n_layers=1,
    n_latent=15,
    n_cell_types=n_cell_types,
    ce_weight=10,
)

compovae.train(
    max_epochs=1,
    train_size=0.8,
    validation_size=0.2,
    plan_kwargs=dict(n_epochs_kl_warmup=80),
    early_stopping=False,
)

adata.obsm["ct_pred"] = compovae.predict_neighborhood(softmax=True)

INFO     Using column names from columns of adata.obsm['neighborhood_composition']                                 


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1: 100%|██████████| 1/1 [00:03<00:00,  3.95s/it, v_num=1, train_loss_step=195, train_loss_epoch=209]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|██████████| 1/1 [00:03<00:00,  3.95s/it, v_num=1, train_loss_step=195, train_loss_epoch=209]
